In [8]:
import math
p = 59
q = 53

n = p * q
phi = (p-1)*(q-1)
e = 2
while e < phi:
	if math.gcd(e, phi) == 1:
		break
	else:
		e+=1
print("E generated")
#private key
d = 0
for i in range(1, phi):
	if (1 + i*phi)%e == 0:
		d = int((1 + i*phi)/e)
		break
	i+=1

print("E = (",e, ", ", n, ")" )
print("D = (",d, ", ", n, ")" )
plaintext = int(input("Enter plaintext"))
cipher = (plaintext**e)%n
print("plaintext = ", plaintext)
print("ciphertext = ", cipher)
decrpyted_text = (cipher**d)%n
print("decrypted text = ", decrpyted_text)

E generated
E = ( 3 ,  3127 )
D = ( 2011 ,  3127 )
plaintext =  21
ciphertext =  3007
decrypted text =  21


In [14]:
def isPrime(n):
	for i in range(2, int(math.sqrt(n)) + 1):
		if(n%i == 0):
			return False
	return True


False

In [4]:
def primitiveRoot(n):
	ans = []
	for i in range(1, n):
		data = []
		flag = False
		for x in range(1, n):
			res = (i**x)%n
			# print("i: ", i, " x: ", x, " res: ", res)
			if res in data or res == 0:
				# print("breaking")
				flag = True
				break
			data.append(res)
		if not flag:
			ans.append(i)
	return ans

primitiveRoot(11)

[2, 6, 7, 8]

In [5]:
p = int(input("p = "))
print("Primitive roots of p are ", primitiveRoot(p))
g = int(input("Select G from above list"))
privateA = int(input())
privateB = int(input())
x = (g**privateA)%p
y = (g**privateB)%p
ka = (y**privateA)%p
kb = (x**privateB)%p
print("x = ", x)
print("y = ", y)
print("ka = ", ka)
print("kb = ", kb)

if(ka == kb):
	print("keys matched\n")

Primitive roots of p are  [2, 6, 7, 11]
x =  3
y =  6
ka =  9
kb =  9
keys matched



In [15]:
import sys
import math
P = 59

def moddiv(a, b, m=P):
	return modmul(a, pow(b, m - 2, m))

def modmul(a, b, m = P):
	return ((a%m)*(b%m))%m

def getpositive(a, m=P):
	a = a%m
	a += m
	a = a%m
	return a

class Point:
	def __init__(self, x, y):
		self.x = x
		self.y = y
	
	def __str__(self):
		return f"({self.x}, {self.y})"

	def __eq__(self, p):
		return self.x == p.x and self.y == p.y
	
class Curve:
	def __init__(self, a, b):
		self.a = a
		self.b = b
	
	def add(self, p1, p2):
		l = 0
		if p1 == p2:
			num = 3*pow(p1.x, 2) + self.a
			den = 2*p1.y
		else:
			num = p2.y - p1.y
			den = p2.x - p1.x
		l = moddiv(num, den)
		x3 = l*l - p1.x - p2.x
		y3 = l*(p1.x - x3) - p1.y
		x3 = getpositive(x3)
		y3 = getpositive(y3)
		return Point(x3, y3)

	def mul(self, k, p1):
		temp = p1
		while k!=1:
			temp = self.add(temp, p1)
			k -= 1
		return temp
	
	def sub(self, p1, p2):
		# newy = getpositive(-1*p2.y)
		newpoint = Point(p2.x, -p2.y)
		return self.add(p1, newpoint)

	def get_points(self):
		for x in range(0, P):
			y = math.sqrt(x**3 + self.a*x + self.b)
			if(y == int(y)):
				print(f"({x}, {int(y)%P})")


def encrypt(message, publickey, G, curve):
	k = 5
	return [curve.mul(k,G), curve.add(message, curve.mul(k,publickey))]

def decrypt(cipher, privatekey, curve):
	return curve.sub(cipher[1], curve.mul(privatekey, cipher[0]))


a = 2
b = 4

if(4*(a**3) + 27*(b**2))%P == 0:
	print("Take other values for a and b")
	sys.exit()

c = Curve(a, b)
c.get_points()

G = Point(0 , 2)

privatekey = 7
publickey = c.mul(privatekey, G)

plaintext = Point(10, 32)

cipher = encrypt(plaintext, publickey, G, c)

print(cipher[0])
print(cipher[1])

decrypted_text = decrypt(cipher, privatekey, c)
print(decrypted_text)

(0, 2)
(2, 4)
(7, 19)
(10, 32)
(28, 8)
(2, 55)
(10, 32)


In [110]:
import pandas as pd
import numpy as np

df = pd.read_csv('./sales.csv')
train_df = df[:-1].copy()
test_df = df[-1:].copy()
x_train = train_df.drop("Buys", axis=1)
y_train = np.array(train_df['Buys'])

class Node:
	def __init__(self, feature, values):
		self.feature = feature
		self.values = values
		self.yes = None
		self.no = None

class DecisionTree:
	def __init__(self):
		self.tree = None
	
	def fit(self, x, y):
		self.get_unique_feature_values(x)
		self.tree = self.build_tree(x, y)
	
	def get_unique_feature_values(self, x):
		self.unique_feature_values = {}
		for feature in x.columns:
			self.unique_feature_values[feature] = np.unique(list(x[feature]))
		# print(self.unique_feature_values)

	def build_tree(self, x, y, parent_feature="",parent_impurity=100):
		if(len(x) == 0):
			return None
		best_feature, best_values, impurity = self.select_best_feature(x, y, parent_feature)
		print(best_feature, best_values, impurity)
		if impurity >= parent_impurity:
			return None
		
		node = Node(best_feature, best_values)
		
		x_yes, y_yes = self.filter_data(x, y, best_feature, best_values, True)
		x_no, y_no = self.filter_data(x, y, best_feature, best_values, False)

		node.yes = self.build_tree(x_yes, y_yes, best_feature, impurity)
		node.no = self.build_tree(x_no, y_no, best_feature, impurity)

		if node.yes is None: node.yes = True
		if node.no is None: node.no = False
		return node

	def select_best_feature(self, x, y, parent_feature):
		best_impurity = 100
		for feature in x.columns:
			values, impurity = self.get_feature_impurity(x, y, feature)
			if impurity <= best_impurity and feature != parent_feature:
				best_impurity = impurity
				best_values = values
				best_feature = feature
		return best_feature, best_values, best_impurity

	def get_feature_impurity(self, x, y, feature):
		values = self.unique_feature_values[feature]
		n = len(values)**2 - 1
		best_impurity = 100
		for i in range(1, n):
			# print(i)
			idx = self.parse(i, len(values))
			# print(idx)
			values_subset = values[idx].copy()
			# print(values_subset)
			impurity = self.get_impurity(x, list(y), feature, values_subset)
			# print(impurity)
			if impurity <= best_impurity:
				best_impurity = impurity
				best_values = values_subset
		return best_values, best_impurity

	def parse(self, xt, n):
		b = bin(xt)[2:]
		indexstring = (n - len(b)) * '0' + b
		# print(indexstring)
		return [i for i in range(0, len(indexstring)) if indexstring[i] == '1']

	def get_impurity(self, x, y, feature, values):
		total_yes = [0, 0]
		total_no = [0, 0]
		for i in range(0, x.shape[0]):
			if x[feature][i] in values:
				if y[i]=='Yes':
					total_yes[1] += 1
				else:
					total_yes[0] += 1
			else:
				if y[i]=='Yes':
					total_no[1] += 1
				else:
					total_no[0] += 1
		# print(x)
		return self.gini(total_yes, total_no)

	def gini(self, total_yes, total_no):
		yes_count = total_yes[0] + total_yes[1]
		no_count = total_no[0] + total_no[1]

		try:
			gini_yes = 1 - pow(total_yes[0]/yes_count, 2) - pow(total_yes[1]/yes_count, 2)
		except: gini_yes = 0

		try:
			gini_no = 1 - pow(total_no[0]/no_count, 2) - pow(total_no[1]/no_count, 2)
		except: gini_no = 0

		return (yes_count*gini_yes + no_count*gini_no)/(yes_count + no_count)

	def filter_data(self, x, y, feature, values, flag):
		x_filtered = x[x[feature].isin(values) == flag].copy()
		idx = list(x_filtered.index)
		x_filtered = x_filtered.reset_index().drop([ 'index'], axis=1)
		y_filtered = y[idx].copy()
		return x_filtered, y_filtered

	def predict(self, x):
		ypred = []
		for i in range(len(x)):
			ypred.append(self.make_prediction(x.iloc[i], self.tree))
		return ypred

	def make_prediction(self, x, node):
		if type(node) == bool:
			return node
		value = x[node.feature]
		if value in node.values: node = node.yes
		else: node = node.no
		return self.make_prediction(x, node)

	def preorderprint(self, node, level):
		if type(node) == bool:
			print(node)
			return
		print(node.feature, node.values)
		print("no of ", node.feature, " level = ", level)
		self.preorderprint(node.no, level + 1)
		print("yes of ", node.feature, " level = ", level)
		self.preorderprint(node.yes, level + 1)

tree = DecisionTree()
tree.fit(x_train, y_train)
x_test = test_df.drop("Buys", axis = 1)
print(tree.predict(x_test))

print(f'Root: {tree.tree.feature}, Values: {tree.tree.values}')

tree.preorderprint(tree.tree, 0)
x_test

Age ['21-35' '>35'] 0.31923076923076926
MaritialStatus ['Married'] 0.16666666666666666
Age ['21-35' '<21'] 0.0
MaritialStatus ['Married'] 0.0
MaritialStatus ['Married'] 0.0
Gender ['Female'] 0.0
MaritialStatus ['Married'] 0.0
MaritialStatus ['Married'] 0.0
Gender ['Female'] 0.0
MaritialStatus ['Married'] 0.0
MaritialStatus ['Married'] 0.0
[False]
Root: Age, Values: ['21-35' '>35']
Age ['21-35' '>35']
no of  Age  level =  0
Gender ['Female']
no of  Gender  level =  1
False
yes of  Gender  level =  1
True
yes of  Age  level =  0
MaritialStatus ['Married']
no of  MaritialStatus  level =  1
Gender ['Female']
no of  Gender  level =  2
False
yes of  Gender  level =  2
True
yes of  MaritialStatus  level =  1
Age ['21-35' '<21']
no of  Age  level =  2
False
yes of  Age  level =  2
True


,Age,Income,Gender,MaritialStatus
13,>35,Medium,Male,Married
